In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

In [2]:
class Layers:
    def __init__(self):
        self.params = []  # Parametros
        self.grads = []  # Gradiantes
        
    def __call__(self, x):
        return x
    
    def backward(self, grad):
        return grad
    
    def update(self, params):
        '''Si hay parámetros, los actualizaremos con
        el resultado que indique el optimizer'''
        return

In [ ]:
class Linear(Layer):
    def __init__(self, dim_in, dim_out):
        self.weigths = np.random.normal(loc=0.0,
                                       scale=np.sqrt(2 / (dim_in + dim_out)),
                                       size=(dim_in, dim_out))
        self.bias = np.zeros(dim_out)
        
    def __call__(self, x):
        self.x = x
        self.params = [self.weights, self.bias]
        return np.dot(x, self.weigths) + self.bias  # Salida del perceptrón
    
    def backward(self, grad_output):
        # gradiantes para la capa siguiente
        grad = np.dot(grad_output, self.weigths.T)
        self.grad_weigths = np.dot(self.x.T, grad_output)
        
        # gradiantes para actualizar pesos
        self.grad_bias = grad_output.mean(axis=0) * self.x.shape[0]
        self.grads = [self.grad_weigths, self.grad_bias]
        return grad
    
    def update(self, params):
        self.weigths, self.bias = params

In [3]:
class Tanh(Activation):
    def activation(self, x):
        return np.tanh(x)
    
    def prime(self, x):
        return 1 - np.tanh(x) ** 2 

In [4]:
class Sigmoide(Activation):
    def activation(self, x):
        return 1 / (1 + np.exp(-x))
    
    def prime(self, x):
        sigmoide = self.activation(x)
        return sigmoide * (1 - sigmoide)

In [5]:
class ReLu(Activation):
    def activation(self, x):
        return np.maximum(0, x)
    
    def prime(self, x):
        return x > 0

In [ ]:
class MLP:
    def __init__(self, layers):
        self.layers = layers
    
    def __call__(self, x):
        '''Calculamos la salida del modelo aplicando
        cada capa de manera secuencial
        '''
        for layer in self.layers:
            x = layers(x)
        return x